In [1]:
import os
import sparse
import random
import math
from scipy.spatial.distance import hamming
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
from datetime import timedelta

from HQA import HQA, lookahead

In [2]:
def count_non_local_comms(Ps, distance_matrix=None):
    if distance_matrix == None:
        distance_matrix = [[1 if j != i else 0 for j in range(N)] for i in range(N)]

    comms = []
    for i in range(1, len(Ps)):
        slice_comms = 0
        for q in range(len(Ps[i])):
            slice_comms += distance_matrix[Ps[i-1][q]][Ps[i][q]]
        comms.append(slice_comms)

    return comms

In [3]:
slices_file = 'random_q200_s1800_2qbf083_1.npz'
Gs = sparse.load_npz(slices_file)
qubits = Gs.shape[1]

print(Gs.shape)

(64, 200, 200)


In [4]:
def oee(A, G, N, part=None):
    '''
    :param A: Weight matrix (Chong's weights)
    :param G: Timeslice-Interaction graph (binary)
    :param N: Num of partitions
    :param part: Initial partition, generates random if None

    :return: OEE solution
    '''
    
    if isinstance(A, sparse.COO):
        A = A.todense()
    
    n_nodes = A.shape[0]
    n_per_part = int(n_nodes / N)

    if part is None:
        part = [i for i in range(N) for _ in range(n_per_part)]
        random.shuffle(part)

    g_max = 1
    swaps = 0

    swapped = []

    # Step 7
    while g_max > 0:
        # Step 1
        C = [i for i in range(n_nodes)]
        index = 0

        W = np.zeros([n_nodes, N])
        D = np.empty([n_nodes, N])

        # Precompute partitions
        P = np.stack([A[np.where(np.array(part) == i)[0]] for i in range(N)])
        
        for i in range(n_nodes):
            for l in range(N):
                W[i, l] = np.sum(P[l, :, i])
        
        for i in range(n_nodes):
            for l in range(N):
                D[i, l] = W[i, l] - W[i, part[i]]

        g = []
        # Step 4
        while len(C) > 1:

            # Step 2
            g.append([-np.inf, None, None])
            for i in C:
                for j in C:
                    g_aux = D[i, part[j]] + D[j, part[i]] - 2*A[i, j]
                    if g_aux > g[index][0]:
                        g[index][0] = g_aux
                        g[index][1] = i
                        g[index][2] = j
            
            a = g[index][1]
            b = g[index][2]

            C.remove(g[index][1])
            if g[index][1] != g[index][2]:
                C.remove(g[index][2])

            # Step 3
            for i in C:
                for l in range(N):
                    if l == part[a]:
                        if part[i] != part[a] and part[i] != part[b]:
                            D[i, l] = D[i, l] + A[i, b] - A[i, a]
                        if part[i] == part[b]:
                            D[i, l] = D[i, l] + 2*A[i, b] - 2*A[i, a]
                    elif l == part[b]:
                        if part[i] != part[a] and part[i] != part[b]:
                            D[i, l] = D[i, l] + A[i, a] - A[i, b]
                        if part[i] == part[a]:
                            D[i, l] = D[i, l] +2*A[i, a] - 2*A[i, b]
                    else:
                        if part[i] == part[a]:
                            D[i, l] = D[i, l] + A[i, a] - A[i, b]
                        elif part[i] == part[b]:
                            D[i, l] = D[i, l] + A[i, b] - A[i, a]
                    
            index += 1
        g_max = np.cumsum([i[0] for i in g])
        m = np.argmax(g_max)
        g_max = g_max[m]

        for i in g[:m+1]:
            # print(f'Swapping nodes {i[1]} and {i[2]}')
            swaps += 1
            part[i[1]], part[i[2]] = part[i[2]], part[i[1]] # Swap
            swapped.append([i[1], i[2]])
            # print(swapped[i[1]][i[2]])
    
    return part, swaps, swapped

In [5]:
# cores = [1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 30, 60]
# TODO: explodes with 40 cores
cores = [10, 20]

In [6]:
HQA_comms = []

for N in cores:
    part = [i for i in range(N) for _ in range(int(qubits/N))]
    random.shuffle(part)

    Ps = np.zeros((Gs.shape[0]+1, Gs.shape[1]), dtype=int)
    Ps[0] = part

    ##### Modifying initial partition (OEE instead of random assignation)
    Ps[0], _, _ = oee(lookahead(Gs), None, N, part)

    all_to_all_topology = None
    Ps_HQA = HQA(Gs, Ps.copy(), N, qubits, qubits, distance_matrix=all_to_all_topology)

    non_local_comunications = sum(count_non_local_comms(Ps_HQA[1:], distance_matrix=all_to_all_topology))
    print(f"Non-local communications: {non_local_comunications}")
    HQA_comms.append(sum(count_non_local_comms(Ps_HQA[1:])))


100%|██████████| 64/64 [00:29<00:00,  2.14it/s]


Non-local communications: 1403


100%|██████████| 64/64 [00:32<00:00,  1.98it/s]

Non-local communications: 1708


In [7]:
HQA_comms = []

for N in cores:
    part = [i for i in range(N) for _ in range(int(qubits/N))]
    random.shuffle(part)

    Ps = np.zeros((Gs.shape[0]+1, Gs.shape[1]), dtype=int)
    Ps[0] = part

    ##### Modifying initial partition (OEE instead of random assignation)
    Ps[0], _, _ = oee(lookahead(Gs), None, N, part)

    all_to_all_topology = None
    Ps_HQA = HQA(Gs, Ps.copy(), N, qubits, qubits, distance_matrix=all_to_all_topology)

    non_local_comunications = sum(count_non_local_comms(Ps_HQA[1:], distance_matrix=all_to_all_topology))
    print(f"Non-local communications: {non_local_comunications}")
    HQA_comms.append(sum(count_non_local_comms(Ps_HQA[1:])))


100%|██████████| 64/64 [00:30<00:00,  2.09it/s]


Non-local communications: 1393


100%|██████████| 64/64 [00:32<00:00,  1.96it/s]

Non-local communications: 1702


In [8]:
print(Ps[0])

[ 2  1  0  0  8  1  7 12  3 18  6 19 13 17 14 16  9 13 13 17  5  5 16  2
  2  4  7 10  8  5  5 18  6 13 18 10  6 19 15 11 12  4 12  3  3 19  6  7
 16 16  3  6  8  4 15  6  9  4  9  8  8  1 13 18 12 14 19 19  6  6 17 15
  7 16  9 18  2 19  0 14  1 15  7 11  6 18  2 10  4 10 14 16 11 12  7  9
  3  7  9 18 17 17 14  5 16  5 11  0  1 11  3 10  2  8 13 11 15  0  5 19
  5 17  2 10 16 15  8  2 13  1 11  5 17 17  1 14  6  8  8 19  3 12  0  0
 14  7 11  1 12  5 13  9 13  0  9  1  7 15 16  2 15 14 15 19  4 12 17  4
  4  3 18  9  8 10  3  0 18 14  7 15  0 13 16 10 12 18  9 10 11  3  2 17
 12  4 14 11  4 10 19  1]


In [9]:
HQA_comms = []

for N in cores:
    part = [i for i in range(N) for _ in range(int(qubits/N))]
    random.shuffle(part)

    Ps = np.zeros((Gs.shape[0]+1, Gs.shape[1]), dtype=int)
    Ps[0] = part

    ##### Modifying initial partition (OEE instead of random assignation)
    Ps[0], _, _ = oee(lookahead(Gs), None, N, part)

    # 0-1-2-3-4-5-6-7-8-9
    line_topology = [[abs(i-j) for j in range(N)] for i in range(N)]
    Ps_HQA = HQA(Gs, Ps.copy(), N, qubits, qubits, distance_matrix=line_topology)

    print(sum(count_non_local_comms(Ps_HQA[1:], distance_matrix=line_topology)))
    HQA_comms.append(sum(count_non_local_comms(Ps_HQA[1:])))

100%|██████████| 64/64 [00:29<00:00,  2.17it/s]


5372


100%|██████████| 64/64 [00:31<00:00,  2.02it/s]

12528
